In [1]:
import tensorflow as tf

In [2]:
cifar10 = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

val_images = train_images[-10000:]
val_labels = train_labels[-10000:]
train_images = train_images[:-10000]
train_labels = train_labels[:-10000]

print(train_labels.shape)
print(val_labels.shape)

(40000, 1)
(10000, 1)


In [3]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9]
b = a[-3:] # last 3 elements
print(b)
a = a[:-3] # all but the last 3 elements
print(a)

[7, 8, 9]
[1, 2, 3, 4, 5, 6]


<img src="https://forums.fast.ai/uploads/default/original/2X/7/75749aef7b40c070627de90f916bb61ac93037a7.png" />

In [4]:
model = tf.keras.Sequential()
# A lambda layer to resize the image
model.add(tf.keras.layers.Lambda( 
    lambda image: tf.image.resize( 
        image, 
        (224, 224), 
        method = tf.image.ResizeMethod.BICUBIC,
        preserve_aspect_ratio = True
    )
))

# VGG-16 layers
layers = [
          tf.keras.layers.Conv2D(64,3, padding="SAME", activation="relu"),
          tf.keras.layers.Conv2D(64,3, padding="SAME", activation="relu"),
          tf.keras.layers.MaxPool2D(2),
        
          tf.keras.layers.Conv2D(128,3, padding="SAME", activation="relu"),
          tf.keras.layers.Conv2D(128,3, padding="SAME", activation="relu"),
          tf.keras.layers.MaxPool2D(2),
        
          tf.keras.layers.Conv2D(256,3, padding="SAME", activation="relu"),
          tf.keras.layers.Conv2D(256,3, padding="SAME", activation="relu"),
          tf.keras.layers.Conv2D(256,3, padding="SAME", activation="relu"),
          tf.keras.layers.MaxPool2D(2),
        
          tf.keras.layers.Conv2D(512,3, padding="SAME", activation="relu"),
          tf.keras.layers.Conv2D(512,3, padding="SAME", activation="relu"),
          tf.keras.layers.Conv2D(512,3, padding="SAME", activation="relu"),
          tf.keras.layers.MaxPool2D(2),
        
          tf.keras.layers.Conv2D(512,3, padding="SAME", activation="relu"),
          tf.keras.layers.Conv2D(512,3, padding="SAME", activation="relu"),
          tf.keras.layers.Conv2D(512,3, padding="SAME", activation="relu"),
          tf.keras.layers.MaxPool2D(2),
        
          tf.keras.layers.Flatten(),
        
          tf.keras.layers.Dense(4096),
          tf.keras.layers.Dense(4096),
          tf.keras.layers.Dense(10, activation="softmax")
        ]

# Add the rest of the layers
# So we have lambda layer (resize the image) -> VGG-16 layers -> output
for layer in layers:
  model.add(layer)

# Call once with dummy data to build the model so you can get the summery
inp = tf.random.normal((1, 224, 224, 3))
print(model(inp))
print(model.summary())

tf.Tensor(
[[0.1011761  0.09884353 0.09808569 0.09969622 0.09943717 0.10092881
  0.10124676 0.1010641  0.10075877 0.09876293]], shape=(1, 10), dtype=float32)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (1, 224, 224, 3)          0         
                                                                 
 conv2d (Conv2D)             (1, 224, 224, 64)         1792      
                                                                 
 conv2d_1 (Conv2D)           (1, 224, 224, 64)         36928     
                                                                 
 max_pooling2d (MaxPooling2D  (1, 112, 112, 64)        0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (1, 112, 112, 128)        73856     
                              

In [5]:
# Prepare the model for training
model.compile(optimizer='sgd', # Stochastic gradient descent optimiser
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), # Crossentropy loss
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]) # Accuracy measure

In [6]:
# Train the model
history = model.fit(
  x=train_images, # Model inputs
  y=train_labels, # True labels for error
  batch_size=32,  # Training batch size; How many samples per time we feed in
  epochs=10,      # Number of epochs, or how many times we go trough the entire dataset to update the parameters
  validation_data=(val_images, val_labels) # Validation inputs and true lables
)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1250/1250 [==============================] - 1142s 901ms/step - loss: 2.2708 - sparse_categorical_accuracy: 0.1348 - val_loss: 2.3248 - val_sparse_categorical_accuracy: 0.1022
Epoch 2/10
1250/1250 [==============================] - 1130s 904ms/step - loss: 1.9221 - sparse_categorical_accuracy: 0.3067 - val_loss: 1.7601 - val_sparse_categorical_accuracy: 0.3717
Epoch 3/10
1250/1250 [==============================] - 1125s 900ms/step - loss: 1.5907 - sparse_categorical_accuracy: 0.4349 - val_loss: 1.5394 - val_sparse_categorical_accuracy: 0.4553
Epoch 4/10
1250/1250 [==============================] - 1128s 902ms/step - loss: 1.3737 - sparse_categorical_accuracy: 0.5178 - val_loss: 1.2820 - val_sparse_categorical_accuracy: 0.5416
Epoch 5/10
1250/1250 [==============================] - 1129s 904ms/step - loss: 1.1762 - sparse_categorical_accuracy: 0.5861 - val_loss: 1.1172 - val_sparse_categorical_accuracy: 0.6126
Epoch 6/10
1250/1250 [==============================] - 1130s 904ms/step - l